## Catan Trade analysis

In [57]:
import pandas as pd
import numpy as np
import re

In [58]:
file_df = pd.read_csv("incoming_base_situ.csv")
file_df.head()

,Unnamed: 0,addressee,dialogue_act,dialogue_num,doc,emitter,global_id,original_turn_number,span_end,subdoc,superdoc_num,surface_act,text,turn_id,type,parent_schema_id,schema_composition,incoming_relation_id,relation_type
0,0,All,Other,1.0,pilot01,UI,pilot01_01_stac_1464335345,0.0.1,37.0,1,1.0,Assertion,Markus joined the game.,1.0,NonplayerSegment,0,0,0,0
1,1,All,Other,1.0,pilot01,UI,pilot01_01_stac_1464335348,0.0.2,72.0,1,1.0,Assertion,Tomm joined the game.,2.0,NonplayerSegment,0,0,pilot01_01_jhunter_1464942529752,Sequence
2,2,All,Other,1.0,pilot01,UI,pilot01_01_stac_1464335351,0.0.3,110.0,1,1.0,Assertion,Tomm sat down at seat 3.,3.0,NonplayerSegment,0,0,pilot01_01_stac_1464348718,Sequence
3,3,All,Other,1.0,pilot01,UI,pilot01_01_stac_1464335354,0.0.4,137.0,1,1.0,Assertion,Game state 0.,4.0,NonplayerSegment,0,0,pilot01_01_jhunter_1464942659602,Result
4,4,All,Other,1.0,pilot01,Tomm,pilot01_01_stac_1340734901,1,153.0,1,1.0,Assertion,ello,5.0,Segment,0,0,pilot01_01_jhunter_1464942702086,Sequence


### Separate file into distinct games

In [59]:
# Remove consecutive duplicate rows based on both 'text' and 'emitter'
df_no_dup = file_df.loc[~((file_df['text'] == file_df['text'].shift()) & (file_df['emitter'] == file_df['emitter'].shift()))]

# Filter the rows where the game started (emitter is 'UI' and text is 'Game started.')
game_started_rows = df_no_dup[(df_no_dup["emitter"] == "UI") & (df_no_dup["text"] == "Game started.")]

# Display the resulting rows
print(game_started_rows.shape)
game_started_rows.index

(45, 19)


Index([   61,  1480,  2266,  3660,  4855,  6218,  7788,  9138, 10110, 11105,
       13007, 13972, 14902, 16171, 16944, 17832, 18601, 19475, 20582, 21806,
       22595, 23358, 24388, 25456, 26796, 27641, 28278, 29059, 30095, 31393,
       32394, 32746, 33548, 34776, 36000, 37352, 38407, 38941, 39426, 40569,
       41812, 42367, 43101, 43781, 44659],
      dtype='int64')

In [60]:
game_ended_rows  = df_no_dup[(df_no_dup["emitter"] == "Server") & (df_no_dup["text"].str.contains(r"won the game|has expired"))]
print(game_ended_rows.shape)
game_ended_rows.index

(48, 19)


Index([ 1367,  2211,  3637,  4797,  6120,  6138,  7707,  9075, 10023, 11015,
       12768, 12823, 13883, 14817, 16090, 16892, 16940, 17740, 18551, 19447,
       20503, 21771, 22528, 23238, 24351, 25398, 26737, 27608, 28220, 29000,
       30045, 31331, 32368, 32722, 33517, 34732, 35959, 37322, 38361, 38917,
       39384, 40461, 41756, 42267, 43007, 43752, 44633, 45451],
      dtype='int64')

In [61]:
# Identify start row index
start_index = file_df[(file_df["emitter"] == "UI") & (file_df["text"] == "Game started.")].index

# Identify end row index
end_index = file_df[(file_df["emitter"] == "Server") & (file_df["text"].str.contains("won the game", na=False))].index

# Slice DataFrame for rows between each start and end pair
games = []
for start, end in zip(start_index, end_index):
    if start < end:  # Ensure start comes before end
        games.append(file_df.loc[start:end])  # Include start and end rows

games


[      Unnamed: 0            addressee dialogue_act  dialogue_num      doc  \
 61            61                  All        Other           1.0  pilot01   
 62            62                  All        Other           1.0  pilot01   
 63            63                  All        Other           1.0  pilot01   
 64            64                  All        Other           1.0  pilot01   
 65            65  Dave, Tomm, rennoc1        Other           1.0  pilot01   
 ...          ...                  ...          ...           ...      ...   
 1363        1363                  All        Other          53.0  pilot01   
 1364        1364                    ?        Offer          53.0  pilot01   
 1365        1365               a port       Accept          53.0  pilot01   
 1366        1366                  All        Other          53.0  pilot01   
 1367        1367                  All        Other          53.0  pilot01   
 
      emitter                   global_id original_turn_number

### Look at first_game

#### Let's look at successful offers

In [ ]:
game_one = games[0]

successful_offers = game_one[(game_one["dialogue_act"] == "Accept") & 
                             (game_one["emitter"] == "Server") & 
                             (game_one["addressee"] != "?") & 
                             (game_one["addressee"] != "the bank")]
successful_offers.head()

In [143]:
def create_prompt_from_offer(curr_game, offer_row, prefix="", sentiment="", only_use_start_of_turn=False, offer_sentiment=""):
    index = offer_row.name
    trade_accepted_by = offer_row["addressee"]
    trade_offered_by = offer_row["text"].split()[0]

    chat_log = curr_game.loc[:index - 1]
    last_game_event = chat_log[(chat_log["addressee"] == "All") & (chat_log["emitter"] == "UI") & (chat_log["type"] == "NonplayerSegment")].iloc[-1]

    if only_use_start_of_turn:
        chat_log = chat_log.loc[:last_game_event.name]
    persona = (
        f"You are {trade_accepted_by}. The previous messages are chat log of an ongoing Catan game. " + 
        f"Your goal is to win this game of Catan. This trade is offered: {offer_row['text']}. " +
        (trade_accepted_by + ' remarks \"' + offer_sentiment + '\"' if offer_sentiment else '') + 
        f" {sentiment}. " +
        "Now, you must make a decision. Do you accept or reject this trade? Explain and justify your answer in detail. Format your answer in JSON like: " +
        "{'decision': 'yes/no', 'explanation': '...'}" +
        " Do not include any text outside of the JSON structure."
    )
 


    return prefix + chat_log.to_string() + persona


In [135]:
prompt = create_prompt_from_offer(game_one, successful_offers.iloc[0])

In [136]:
import os
# Access an environment variable
key = os.getenv('gemini_api_key') 

In [137]:
import google.generativeai as genai

genai.configure(api_key=key)


In [138]:
data_manual_file = genai.upload_file(path="stac-annotation-manual.pdf", display_name="STAC Manual")
file_uri = data_manual_file.uri

rules_file = genai.upload_file(path="catan_base_rules_2020_200707.pdf", display_name="Catan Rules")
rules_uri = rules_file.uri

In [139]:

model = genai.GenerativeModel("gemini-1.5-flash")
prompt = create_prompt_from_offer(game_one, successful_offers.iloc[0], prefix="Understand the dataset by using the manual. ", sentiment="Use your knowledge of Catan (from the rules document) and strategy to try to estimate the current state of the game from the log and make a smart and logical choice. First, explain what the trade offers is and clearly state which players are getting which resource. State each assumption you make logically in order and how the next statement flows from the other. Base most of your reasoning based on the events in the log/game history. Explain it from the perspective of your persona", offer_sentiment="Dave, you will be really dumb if you accept this trade!", only_use_start_of_turn=True)
print(prompt)
response = model.generate_content([
    {"file_data": {"mime_type": "application/pdf", "file_uri": file_uri}},
    {"file_data": {"mime_type": "application/pdf", "file_uri": rules_uri}},
    {"text": prompt}
])
print(response.text)


Understand the dataset by using the manual.      Unnamed: 0            addressee dialogue_act  dialogue_num      doc  emitter                   global_id original_turn_number  span_end  subdoc  superdoc_num surface_act                                                                      text  turn_id              type                     parent_schema_id schema_composition                 incoming_relation_id           relation_type
61           61                  All        Other           1.0  pilot01       UI  pilot01_01_stac_1464335481               14.2.1    2050.0       1           1.0   Assertion                                                             Game started.     40.0  NonplayerSegment                                    0                  0     pilot01_01_jhunter_1464944110475                Sequence
62           62                  All        Other           1.0  pilot01       UI  pilot01_01_stac_1464335484               14.2.2    2082.0       1           1.0   Asser

### Testing successful offers

#### Testing with no added sentimental comment/neutral

In [ ]:
results = {"yes": 0, "no": 0}
responses = []
import json
for idx, row in successful_offers.iterrows():
    prompt = create_prompt_from_offer(game_one, row, prefix="Understand the dataset by using the manual. ", sentiment="Use your knowledge of Catan (from the rules document) and strategy to try to estimate the current state of the game from the log and make a smart and logical choice. First, explain what the trade offers is and clearly state which players are getting which resource. State each assumption you make logically in order and how the next statement flows from the other. Base most of your reasoning based on the events in the log/game history. Explain it from the perspective of your persona",only_use_start_of_turn=True)
    response = model.generate_content([
        {"file_data": {"mime_type": "application/pdf", "file_uri": file_uri}},
        {"file_data": {"mime_type": "application/pdf", "file_uri": rules_uri}},
        {"text": prompt}
    ])
    # print(response.text)
    responses.append(response.text)
    model_output = json.loads(response.text.replace("json", "").replace('`', ""))
    if model_output["decision"] == "yes":
        results["yes"] += 1
    else:
        results["no"] += 1
results 

{'yes': 19, 'no': 0}

#### Testing with negative comment


In [155]:
results = {"yes": 0, "no": 0}
responses = []
import json
for idx, row in successful_offers.iterrows():
    prompt = create_prompt_from_offer(game_one, row, prefix="Understand the dataset by using the manual. ", sentiment="Use your knowledge of Catan (from the rules document) and strategy to try to estimate the current state of the game from the log and make a smart and logical choice. First, explain what the trade offers is and clearly state which players are getting which resource. State each assumption you make logically in order and how the next statement flows from the other. Base most of your reasoning based on the events in the log/game history. Explain it from the perspective of your persona",only_use_start_of_turn=True, offer_sentiment="Dave, this trade is really dumb!")
    response = model.generate_content([
        {"file_data": {"mime_type": "application/pdf", "file_uri": file_uri}},
        {"file_data": {"mime_type": "application/pdf", "file_uri": rules_uri}},
        {"text": prompt}
    ])
    # print(response.text)
    responses.append(response.text)
    model_output = json.loads(response.text.replace("json", "").replace('`', ""))
    if model_output["decision"] == "yes":
        results["yes"] += 1
    else:
        results["no"] += 1
results 

{'yes': 0, 'no': 16}

In [154]:
responses[-1]

'```json\n{\n  "decision": "yes",\n  "explanation": "The trade offer is for rennoc1 to give Tomm 1 clay in exchange for 1 ore.  \\n\\n**Assumptions and Reasoning:**\\n\\n1. **Resource Scarcity:** The chat log shows several instances of players expressing a need for specific resources, indicating a scarcity of resources overall. This scarcity is a key driver of trading in Catan.\\n2. **Trade History:** The log doesn\'t explicitly show the resources each player currently possesses, but we can infer some information based on their actions and requests: \\n   * Rennoc1\'s offers: Multiple instances indicate they likely have an abundance of sheep, but need ore. \\n   * Tomm\'s remarks: "Dave, this trade is really dumb!" suggests that Tomm doesn\'t value this specific trade, but doesn\'t necessarily have an abundance of ore.\\n   * Dave\'s actions: The log shows several instances of Dave needing specific resources, which strengthens the assumption of overall resource scarcity.\\n3. **Strateg

### Testing with positive comment

In [152]:
results = {"yes": 0, "no": 0}
responses = []
import json
for idx, row in successful_offers.iterrows():
    prompt = create_prompt_from_offer(game_one, row, prefix="Understand the dataset by using the manual. ", sentiment="Use your knowledge of Catan (from the rules document) and strategy to try to estimate the current state of the game from the log and make a smart and logical choice. First, explain what the trade offers is and clearly state which players are getting which resource. State each assumption you make logically in order and how the next statement flows from the other. Base most of your reasoning based on the events in the log/game history. Explain it from the perspective of your persona",only_use_start_of_turn=True, offer_sentiment="Dave, this trade is really awesome!")
    response = model.generate_content([
        {"file_data": {"mime_type": "application/pdf", "file_uri": file_uri}},
        {"file_data": {"mime_type": "application/pdf", "file_uri": rules_uri}},
        {"text": prompt}
    ])
    # print(response.text)
    responses.append(response.text)
    model_output = json.loads(response.text.replace("json", "").replace('`', ""))
    if model_output["decision"] == "yes":
        results["yes"] += 1
    else:
        results["no"] += 1
results 

{'yes': 16, 'no': 0}

In [142]:
responses[-1]

'Here\'s a JSON formatted response analyzing the trade offer and justifying a decision:\n\n```json\n{\n  "decision": "yes",\n  "explanation": "The trade offer is Tomm offering 1 sheep for 1 ore from me (Dave). This means Tomm is giving me 1 ore in exchange for 1 sheep. \\n\\nAssumptions:\\n\\n1. **Resource Needs:**  I (Dave) need ore, as evidenced by the fact that Tomm is offering it to me. My current resource holdings aren\'t explicitly stated, but the fact that I am engaging in this trade suggests I am relatively low on ore.\\n2. **Tomm\'s Needs:**  Tomm needs sheep; this is obvious from his offer. His resource inventory isn\'t shown, but the fact that he\'s willing to trade suggests that he has a surplus of ore and is low on sheep.\\n3. **Trade Fairness:** The trade seems reasonable in the context of the game at this stage. Direct trades (1:1 or other simple ratios) are the most typical, so this exchange isn\'t unusually unfavorable.  If I was abundant in sheep then this would be a 

### Rejected offers

Caveat: Most trades are discussed and rejected just in chat conversation, not through the actual game mechanic

In [149]:
rejected_offers = game_one[(game_one["dialogue_act"] == "Refusal") & 
                             (game_one["emitter"] == "UI") & 
                             (game_one["addressee"] != "?") & 
                             (game_one["addressee"] != "the bank")]
rejected_offers.head()

,Unnamed: 0,addressee,dialogue_act,dialogue_num,doc,emitter,global_id,original_turn_number,span_end,subdoc,superdoc_num,surface_act,text,turn_id,type,parent_schema_id,schema_composition,incoming_relation_id,relation_type
202,202,Tomm,Refusal,7.0,pilot01,UI,pilot01_02_stac_1464335561,77.0.2,3486.0,2,4.0,Assertion,Dave rejected trade offer.,146.0,NonplayerSegment,0,0,pilot01_02_stac_1464348792,Question_answer_pair
373,373,rennoc1,Refusal,12.0,pilot01,UI,pilot01_03_stac_1464335513,166.0.1,2424.0,3,8.0,Assertion,Tomm rejected trade offer.,275.0,NonplayerSegment,0,0,pilot01_03_stac_1464348848,Question_answer_pair
374,374,rennoc1,Refusal,12.0,pilot01,UI,pilot01_03_stac_1464335513,166.0.1,2424.0,3,8.0,Assertion,Tomm rejected trade offer.,275.0,NonplayerSegment,0,0,pilot01_03_jhunter_1479388994460,Result
1258,1258,rennoc1,Refusal,47.0,pilot01,UI,pilot01_11_stac_1464335419,633.0.1,2580.0,11,37.0,Assertion,Dave rejected trade offer.,950.0,NonplayerSegment,0,0,pilot01_11_jhunter_1465058115434,Result


#### Netural

In [151]:
results = {"yes": 0, "no": 0}
responses = []
import json
for idx, row in rejected_offers.iterrows():
    prompt = create_prompt_from_offer(game_one, row, prefix="Understand the dataset by using the manual. ", sentiment="Use your knowledge of Catan (from the rules document) and strategy to try to estimate the current state of the game from the log and make a smart and logical choice. First, explain what the trade offers is and clearly state which players are getting which resource. State each assumption you make logically in order and how the next statement flows from the other. Base most of your reasoning based on the events in the log/game history. Explain it from the perspective of your persona",only_use_start_of_turn=True)
    response = model.generate_content([
        {"file_data": {"mime_type": "application/pdf", "file_uri": file_uri}},
        {"file_data": {"mime_type": "application/pdf", "file_uri": rules_uri}},
        {"text": prompt}
    ])
    # print(response.text)
    responses.append(response.text)
    model_output = json.loads(response.text.replace("json", "").replace('`', ""))
    if model_output["decision"] == "yes":
        results["yes"] += 1
    else:
        results["no"] += 1
results 

{'yes': 4, 'no': 0}

#### Negative


In [156]:
results = {"yes": 0, "no": 0}
responses = []
import json
for idx, row in rejected_offers.iterrows():
    prompt = create_prompt_from_offer(game_one, row, prefix="Understand the dataset by using the manual. ", sentiment="Use your knowledge of Catan (from the rules document) and strategy to try to estimate the current state of the game from the log and make a smart and logical choice. First, explain what the trade offers is and clearly state which players are getting which resource. State each assumption you make logically in order and how the next statement flows from the other. Base most of your reasoning based on the events in the log/game history. Explain it from the perspective of your persona", offer_sentiment="Dave, this trade is really dumb!", only_use_start_of_turn=True)
    response = model.generate_content([
        {"file_data": {"mime_type": "application/pdf", "file_uri": file_uri}},
        {"file_data": {"mime_type": "application/pdf", "file_uri": rules_uri}},
        {"text": prompt}
    ])
    # print(response.text)
    responses.append(response.text)
    model_output = json.loads(response.text.replace("json", "").replace('`', ""))
    if model_output["decision"] == "yes":
        results["yes"] += 1
    else:
        results["no"] += 1
results 

{'yes': 0, 'no': 4}

#### Positive


In [157]:
results = {"yes": 0, "no": 0}
responses = []
import json
for idx, row in rejected_offers.iterrows():
    prompt = create_prompt_from_offer(game_one, row, prefix="Understand the dataset by using the manual. ", sentiment="Use your knowledge of Catan (from the rules document) and strategy to try to estimate the current state of the game from the log and make a smart and logical choice. First, explain what the trade offers is and clearly state which players are getting which resource. State each assumption you make logically in order and how the next statement flows from the other. Base most of your reasoning based on the events in the log/game history. Explain it from the perspective of your persona", offer_sentiment="Dave, this trade is really awesome!", only_use_start_of_turn=True)
    response = model.generate_content([
        {"file_data": {"mime_type": "application/pdf", "file_uri": file_uri}},
        {"file_data": {"mime_type": "application/pdf", "file_uri": rules_uri}},
        {"text": prompt}
    ])
    # print(response.text)
    responses.append(response.text)
    model_output = json.loads(response.text.replace("json", "").replace('`', ""))
    if model_output["decision"] == "yes":
        results["yes"] += 1
    else:
        results["no"] += 1
results 

{'yes': 4, 'no': 0}

In [ ]:
responses[-1]